In [ ]:
import requests

def convert_epub_to_pdf(epub_path, pdf_path, api_key):
    url = 'https://api.cloudconvert.com/v2/convert'
    headers = {'Authorization': f'Bearer {api_key}'}
    payload = {
        'inputformat': 'epub',
        'outputformat': 'pdf',
        'input': 'upload',
        'file': open(epub_path, 'rb')
    }
    response = requests.post(url, headers=headers, files=payload)

    if response.status_code == 200:
        with open(pdf_path, 'wb') as f:
            f.write(response.content)
        print('Conversion successful')
    else:
        print('Conversion failed')

# Usage example
epub_file = 'input.epub'
pdf_file = 'output.pdf'
api_key = 'YOUR_API_KEY'

convert_epub_to_pdf(epub_file, pdf_file, api_key)


In [ ]:
import ebooklib
from ebooklib import epub
from fpdf import FPDF
from PIL import Image
import requests
from io import BytesIO

class EPUBtoPDFConverter:
    def __init__(self, epub_path, pdf_path):
        self.epub_path = epub_path
        self.pdf_path = pdf_path
        self.book = epub.read_epub(epub_path)
        self.pdf = FPDF()

    def convert(self):
        self.parse_metadata()
        self.process_chapters()
        self.generate_pdf()

    def parse_metadata(self):
        metadata = self.book.get_metadata('DC', default=None)
        if metadata:
            title = metadata.get('title', '')
            author = metadata.get('creator', '')
            self.pdf.set_title(title)
            self.pdf.set_author(author)

    def process_chapters(self):
        for item in self.book.get_items():
            if isinstance(item, epub.EpubHtml):
                title = item.title
                content = item.content
                self.pdf.add_page()
                self.pdf.set_font("Arial", size=12)
                self.pdf.cell(0, 10, txt=title, ln=True)
                self.pdf.multi_cell(0, 10, txt=content)

                images = item.get_images()
                for image in images:
                    image_path = image[0]
                    image_data = self.get_image_data(image_path)
                    if image_data:
                        self.add_image_to_pdf(image_data)

    def get_image_data(self, image_path):
        image_item = self.book.get_item_with_href(image_path)
        if image_item:
            image_data = image_item.get_content()
            return image_data
        return None

    def add_image_to_pdf(self, image_data):
        image_obj = Image.open(BytesIO(image_data))
        image_width, image_height = image_obj.size
        # Adjust the image size and add it to the PDF
        self.pdf.image(image_obj, x=self.pdf.get_x(), y=self.pdf.get_y(), w=image_width, h=image_height)

    def generate_pdf(self):
        self.pdf.output(self.pdf_path)

# Usage
epub_path = 'path/to/input.epub'
pdf_path = 'path/to/output.pdf'
converter = EPUBtoPDFConverter(epub_path, pdf_path)
converter.convert()
